In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import recall_score
from tqdm import tqdm
from imblearn.ensemble import BalancedBaggingClassifier
from imblearn.under_sampling import RandomUnderSampler
from sklearn.svm import SVC
from xgboost import XGBClassifier
import pickle
import tensorflow as tf
import math
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing

In [2]:
df = pd.read_csv("../Data_Scrape/Dataset/updated_dataset.csv")

In [3]:
df.head()

,sp_song_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,...,lda_topic_11,lda_topic_12,lda_topic_13,lda_topic_14,lda_topic_15,lda_topic_16,lda_topic_17,lda_topic_18,lda_topic_19,Playlists
0,6edQfeOlqbGteYixpJl3Sm,0.857,0.602,10,-8.311,1,0.0290,0.489000,0.02440,0.3170,...,0.0,0.022993,0.000000,0.122078,0.0,0.000000,0.000000,0.081108,0.386042,[165]
1,5Oe7wHPL4hdEXeF4AOayCi,0.322,0.990,8,-1.785,1,0.1710,0.000044,0.41700,0.0366,...,0.0,0.000000,0.163142,0.022999,0.0,0.144614,0.187207,0.000000,0.000000,[79]
2,6i1uWZYWabNHq2wQnoca58,0.666,0.884,9,-5.243,0,0.0429,0.506000,0.00612,0.0408,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.582131,[15]
3,4BzBtS6PBreni5hNPo2hos,0.609,0.578,9,-7.081,1,0.0414,0.296000,0.00000,0.1500,...,0.0,0.000000,0.511716,0.084905,0.0,0.000000,0.000000,0.115004,0.108140,[168]
4,0GvhHQbWSnGltjl0je61dI,0.699,0.514,4,-12.610,1,0.0315,0.587000,0.02900,0.2200,...,0.0,0.000000,0.000000,0.401954,0.0,0.000000,0.000000,0.000000,0.000000,"[30, 133]"


### Remove songs which are in more than one playlist

In [4]:
dataset = []
data = np.array(df)
for i in df.index:
    l = eval(df["Playlists"][i])
    if len(l) > 1:
        continue
    data[i][-1] = str(l[0])
    dataset.append(data[i])
dataset = np.array(dataset)
        

In [5]:
print(len(df))
print(len(dataset))

11159
6120


In [6]:
counts = []
for i in range(169):
    counts.append(0)
for i in range(len(dataset)):
    counts[int(dataset[i][-1])] += 1
c = 0
playlists_to_keep = set()
for i in range(len(counts)):
    if counts[i] >= 100:
        c += 1
        playlists_to_keep.add(str(i))        
print(c)

13


In [7]:
print(type(dataset))

<class 'numpy.ndarray'>


In [8]:
temp_dataset = []
for c in dataset:
    if str(c[-1]) in playlists_to_keep:
        temp_dataset.append(c)
dataset = temp_dataset
dataset = np.array(dataset)
        

In [9]:
len(dataset)

2249

In [10]:
X = dataset[:, 1:-1]
y = dataset[:, -1]

In [33]:
def run_model(model_type = "LR", neighbours = 5):
    playlists = []

    np.random.seed(123)

    X = dataset[:, 1:-1]
    y = dataset[:, -1]
    
    s = set()
    for c in y:
        s.add(c)
    
    if model_type == 'ANN':
        le = preprocessing.LabelEncoder()
        le.fit(y)
        y = le.transform(y)
        
        enc = OneHotEncoder()
        y = y.reshape(-1, 1)
        enc.fit(y)
        y = enc.transform(y).toarray()
        
    

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10, stratify = y)

    standardScalar = StandardScaler()
    X_train = standardScalar.fit_transform(X_train)
    X_test = standardScalar.transform(X_test)
     
    
    if(model_type == "LR" ):
        model = LogisticRegression()

    elif(model_type == "SVM"):
        model = SVC(kernel = 'linear')

    elif (model_type == "DT"):
        model = DecisionTreeClassifier(criterion = 'entropy', max_depth =  20)

    elif (model_type == "RF"):
        model = RandomForestClassifier(criterion = 'entropy')


    elif(model_type == "XGB"):

        param = {
            "learning_rate" : 0.1,
            "n_estimators" : 500,
            "max_depth" : 20,
            "min_child_weight" : 1,
            "gamma" : 0.1,
            "subsample": 0.9,
            "colsample_bytree" : 0.9,
            "objective" : 'binary:logistic',
            "nthread" : 4,
            "scale_pos_weight" :  1,
            "seed" : 27
        }

        model = XGBClassifier(**param)

    elif(model_type == "KNN"):
        model = KNeighborsClassifier(n_neighbors=neighbours)

    elif(model_type == "ANN"):
        model = Sequential([Dense(units = 64, input_shape = (34, ), activation = 'relu'),
        Dense(units = 32, activation = 'relu'),
        Dense(units = 16, activation = 'relu'),       
        Dense(units = len(s), activation = 'softmax')
        ])
        model.compile(optimizer = tf.keras.optimizers.Adam(lr = 0.05), loss = 'categorical_crossentropy', metrics = ['accuracy'])
        model.summary()




#         xgb = XGBClassifier(**param)    

    if model_type != 'ANN':
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

    else:
        model.fit(X_train, y_train, validation_data = (X_test, y_test), batch_size = 64, epochs = 100)
        y_pred = model.predict_classes(X_test)
#         print(y_test)
        temp = []
        for c in y_test:
            for i in range(len(c)):
                if c[i] == 1:
                    temp.append(i+1)
        y_test = temp
        y_test = np.array(y_test)
    
#     print(y_pred)
#     print(y_test)

    pres_score = precision_score(y_test, y_pred, average = "macro")

    f1= f1_score(y_test, y_pred, average = "macro")
    recall_Score = recall_score(y_test, y_pred, average = "macro")

    result_history = {"Precision Score " : pres_score, "Recall Score " : recall_Score, "F1 Score " : f1 }


    print(result_history)
    return result_history

In [12]:
result_history = run_model(model_type = "XGB")

[19:04:47] WARNING: ../src/learner.cc:516: 
Parameters: { scale_pos_weight } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


['42' '81' '30' '79' '30' '79' '89' '79' '159' '38' '79' '83' '30' '89'
 '123' '38' '30' '85' '85' '42' '83' '83' '42' '79' '123' '38' '123' '123'
 '79' '132' '132' '123' '42' '81' '79' '89' '42' '83' '123' '42' '42'
 '123' '79' '42' '42' '30' '89' '132' '79' '42' '79' '132' '76' '159' '30'
 '132' '89' '42' '79' '123' '79' '79' '89' '42' '76' '42' '79' '30' '81'
 '111' '76' '132' '76' '38' '81' '81' '159' '89' '42' '89' '76' '42' '81'
 '79' '30' '89' '42' '132' '132' '30' '159' '123' '79' '83' '79' '38' '30'
 '38' '83' '83' '30' '42' '159' '123' '42' '79' '30' '111' '81' '81' '89'
 '79' '76' '30' '79' '111' '79' '123' '81' '123' '42' '42' '89' '38' '30'
 '

In [13]:
result_history = run_model(model_type = "RF")

['159' '81' '42' '79' '83' '79' '89' '79' '159' '38' '79' '83' '30' '42'
 '38' '38' '30' '85' '79' '42' '83' '79' '42' '79' '123' '79' '123' '123'
 '79' '123' '42' '89' '42' '81' '79' '89' '42' '83' '123' '42' '42' '123'
 '79' '42' '42' '79' '89' '123' '79' '42' '79' '132' '30' '159' '42' '123'
 '159' '30' '79' '123' '79' '79' '89' '42' '79' '42' '79' '30' '81' '111'
 '132' '132' '76' '38' '30' '81' '159' '89' '42' '89' '30' '42' '81' '79'
 '42' '89' '42' '42' '132' '42' '123' '42' '79' '83' '79' '38' '30' '76'
 '83' '83' '30' '42' '159' '123' '159' '79' '30' '111' '81' '81' '89' '79'
 '76' '42' '79' '42' '79' '123' '81' '123' '30' '42' '89' '38' '42' '42'
 '85' '85' '30' '83' '30' '79' '123' '123' '79' '79' '76' '83' '123' '111'
 '30' '89' '123' '123' '123' '38' '76' '38' '79' '30' '123' '42' '85' '85'
 '79' '79' '79' '132' '79' '83' '79' '42' '42' '85' '42' '42' '42' '111'
 '42' '79' '79' '89' '76' '79' '123' '42' '89' '79' '79' '42' '79' '79'
 '79' '111' '85' '79' '76' '123' '30' '7

In [25]:
result_history = run_model(model_type = "KNN", neighbours = 23)

['111' '81' '38' '79' '30' '79' '89' '79' '132' '38' '79' '83' '30' '89'
 '123' '81' '132' '79' '123' '42' '83' '79' '30' '79' '123' '81' '132'
 '89' '79' '111' '42' '89' '83' '81' '79' '89' '30' '79' '123' '42' '159'
 '132' '79' '42' '42' '38' '89' '89' '79' '42' '79' '132' '123' '42' '42'
 '132' '89' '30' '79' '38' '79' '79' '89' '85' '79' '159' '79' '83' '38'
 '42' '42' '42' '76' '38' '81' '81' '123' '89' '42' '89' '76' '38' '81'
 '79' '42' '79' '42' '159' '132' '159' '123' '132' '79' '83' '123' '38'
 '30' '38' '83' '79' '81' '42' '42' '123' '123' '79' '111' '111' '76' '81'
 '89' '79' '76' '42' '79' '42' '79' '38' '38' '132' '38' '42' '89' '38'
 '42' '132' '79' '85' '38' '111' '123' '79' '123' '42' '79' '79' '76' '79'
 '42' '42' '111' '89' '123' '159' '123' '38' '38' '38' '79' '42' '123'
 '111' '79' '79' '79' '79' '79' '42' '79' '85' '85' '123' '38' '85' '42'
 '89' '42' '123' '42' '79' '79' '89' '76' '123' '89' '42' '89' '79' '79'
 '76' '79' '79' '79' '111' '79' '79' '76' '123' '111

In [15]:
result_history = run_model(model_type = "LR")

['159' '79' '30' '79' '42' '79' '89' '79' '159' '38' '79' '83' '30' '42'
 '38' '38' '30' '83' '83' '42' '83' '83' '111' '85' '123' '38' '42' '123'
 '79' '123' '132' '89' '76' '81' '79' '89' '42' '83' '132' '42' '42' '132'
 '79' '42' '42' '30' '89' '123' '79' '42' '79' '38' '123' '89' '38' '123'
 '89' '30' '79' '123' '79' '79' '89' '83' '85' '81' '79' '159' '38' '42'
 '42' '30' '76' '38' '30' '81' '159' '89' '42' '89' '76' '42' '81' '79'
 '42' '85' '42' '132' '132' '30' '89' '132' '79' '83' '79' '38' '30' '76'
 '83' '83' '81' '42' '42' '42' '123' '79' '30' '111' '38' '81' '89' '79'
 '76' '159' '79' '42' '79' '38' '38' '159' '42' '42' '89' '38' '30' '85'
 '85' '83' '30' '83' '30' '79' '123' '42' '81' '79' '76' '85' '123' '111'
 '42' '123' '159' '42' '123' '38' '76' '38' '79' '42' '123' '111' '85'
 '79' '79' '79' '79' '132' '79' '85' '85' '42' '42' '85' '42' '123' '42'
 '159' '83' '79' '81' '159' '76' '79' '123' '159' '89' '79' '79' '42' '79'
 '76' '85' '123' '85' '79' '76' '123' '30' '81

/home/pankil/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [16]:
result_history = run_model(model_type = "SVM")

['159' '132' '30' '79' '42' '79' '89' '79' '42' '38' '79' '83' '30' '123'
 '132' '38' '30' '83' '79' '132' '79' '85' '111' '85' '42' '79' '42' '83'
 '79' '123' '132' '89' '83' '81' '79' '89' '42' '83' '123' '42' '123'
 '132' '79' '42' '42' '30' '111' '123' '79' '42' '79' '38' '123' '42'
 '132' '123' '159' '30' '85' '123' '79' '79' '89' '83' '85' '159' '79'
 '111' '81' '42' '42' '30' '76' '38' '30' '81' '159' '159' '42' '89' '30'
 '42' '81' '79' '42' '132' '42' '132' '132' '159' '123' '132' '79' '83'
 '79' '38' '30' '76' '83' '83' '81' '42' '42' '42' '123' '79' '111' '111'
 '38' '81' '123' '79' '76' '123' '79' '42' '79' '123' '81' '132' '38' '42'
 '89' '38' '30' '132' '85' '83' '30' '83' '111' '79' '123' '42' '81' '79'
 '159' '85' '42' '111' '42' '123' '123' '42' '123' '38' '38' '38' '79'
 '42' '123' '159' '85' '79' '79' '79' '79' '132' '79' '85' '85' '42' '42'
 '85' '42' '123' '42' '159' '42' '79' '81' '159' '76' '79' '123' '123'
 '89' '79' '79' '42' '79' '76' '85' '111' '85' '79' '76'

In [23]:
result_history = run_model(model_type = "DT")

['89' '81' '42' '79' '79' '79' '123' '85' '159' '38' '81' '83' '76' '123'
 '132' '38' '123' '85' '85' '159' '83' '83' '42' '79' '123' '81' '123'
 '132' '79' '42' '132' '123' '159' '81' '79' '89' '42' '83' '123' '111'
 '42' '159' '79' '132' '30' '83' '89' '123' '79' '42' '79' '42' '76' '123'
 '85' '123' '159' '42' '85' '123' '79' '79' '89' '159' '83' '123' '79'
 '132' '81' '111' '132' '132' '76' '159' '76' '81' '159' '89' '30' '89'
 '123' '42' '81' '79' '42' '159' '30' '42' '42' '30' '132' '42' '79' '83'
 '79' '38' '159' '30' '83' '83' '30' '111' '159' '132' '159' '81' '30'
 '123' '38' '38' '89' '38' '76' '123' '79' '111' '79' '38' '30' '132' '38'
 '30' '89' '38' '123' '30' '76' '83' '38' '83' '123' '79' '123' '123' '79'
 '79' '111' '83' '123' '111' '42' '89' '123' '111' '123' '81' '76' '38'
 '79' '123' '123' '38' '85' '85' '79' '85' '85' '132' '79' '83' '85' '132'
 '30' '85' '42' '123' '83' '85' '83' '79' '81' '89' '76' '30' '38' '42'
 '89' '79' '85' '42' '79' '79' '79' '123' '85' '76'

In [34]:
result_history = run_model(model_type = "ANN")

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_28 (Dense)             (None, 64)                2240      
_________________________________________________________________
dense_29 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_30 (Dense)             (None, 16)                528       
_________________________________________________________________
dense_31 (Dense)             (None, 13)                221       
Total params: 5,069
Trainable params: 5,069
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
29/29 [==============================] - 0s 5ms/step - loss: 2.0045 - accuracy: 0.3224 - val_loss: 1.7296 - val_accuracy: 0.3689
Epoch 2/100
29/29 [==============================] - 0s 2ms/step - loss: 1.7257 - accuracy: 0.3797 - val_loss

/home/pankil/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/pankil/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# f = open("binary_svc_linear_result", "wb")
# pickle.dump(result_history, f)
# f.close()

In [ ]:
# f = open("binary_svc_linear_result", "rb")
# l = pickle.load(f)
# f.close()

In [ ]:
l = {k: v for k, v in sorted(l.items(), key=lambda item: item[1]["F1 Score "], reverse = True)}

In [ ]:
# Top 20 playlists 
for key in list(l.keys())[:20]:
    print(key," = ",  l[key])